# Spark (with PySpark)


In this tutorial, you will learn how to use Apache Spark, a framework for large-scale data processing, within a notebook.
Upon completing this lab you will be able to : 
 + Program in Spark with the Python Language
 + Demonstrate how to read and process data using Spark
 + Compare and contrast RDD and Dataframes. 
 + Build a simple machine learning application with Spark.
 
 
 
# Requirements : Installing Spark, PySpark and configuration to run it on jupyter notebooks

According to your OS, you have to follow the following tutorial :

+ [Windows](https://changhsinlee.com/install-pyspark-windows-jupyter/)
+ [Mac Os X](https://jmedium.com/pyspark-in-mac/)
 
At the end of the installation, you should be able to run the following code that is a kind of Hello word in PySpark.

In [1]:
import findspark
findspark.init('C:\SPARK\spark-2.3.0-bin-hadoop2.7')

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

df = spark.sql('''select 'spark' as hello ''')
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [2]:
spark.stop()

## Part 1 :  Spark Context

When writing a spark program, the first thing to do is to define a `SparkContext`. 

In Spark, communication occurs between a driver and executors. The driver has Spark jobs that it needs to run and these jobs are split into tasks that are submitted to the executors for completion. The results from these tasks are delivered back to the driver.

Here, we will use the `findspark` package that has to be installed using the following command: 

` pip3 install findspark`

Then we can use the `findspark.init()`function to locate the Spark process.




In [3]:
import findspark

findspark.init("C:\SPARK\spark-2.3.0-bin-hadoop2.7")

import pyspark

sc=pyspark.SparkContext("local[*]","First program in Spark")

Try printing out sc to see its type.

In [4]:
sc

<SparkContext master=local[*] appName=First program in Spark>

You can use Python's `dir()` function to get a list of all the attributes (including methods) accessible through the `sc` object.

In [5]:
dir(sc)

['PACKAGE_EXTENSIONS',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accumulatorServer',
 '_active_spark_context',
 '_batchSize',
 '_callsite',
 '_checkpointFile',
 '_conf',
 '_dictToJavaMap',
 '_do_init',
 '_ensure_initialized',
 '_gateway',
 '_getJavaStorageLevel',
 '_initialize_context',
 '_javaAccumulator',
 '_jsc',
 '_jvm',
 '_lock',
 '_next_accum_id',
 '_pickled_broadcast_vars',
 '_python_includes',
 '_repr_html_',
 '_serialize_to_jvm',
 '_temp_dir',
 '_unbatched_serializer',
 'accumulator',
 'addFile',
 'addPyFile',
 'appName',
 'applicationId',
 'binaryFiles',
 'binaryRecords',
 'broadcast',
 'cancelAllJobs',
 '

## Part 2 : Understanding Spark RDD’s

### WordCount in Spark
In this part, we will write the wordcount in Spark and apply it on the novel Dracula of Bram Stocker (from the Gutemberg project).

First read the [pg345.txt](./SparkData/pg345.txt) file.

In [6]:
data = sc.textFile('./SparkData/pg345.txt')
type(data)

pyspark.rdd.RDD

To see the content, of the file, we need to run the action `collect` on the  RDD `data`

In [7]:
data.collect()

['The Project Gutenberg EBook of Dracula, by Bram Stoker',
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.org/license',
 '',
 '',
 'Title: Dracula',
 '',
 'Author: Bram Stoker',
 '',
 'Release Date: August 16, 2013 [EBook #345]',
 '',
 'Language: English',
 '',
 '',
 '*** START OF THIS PROJECT GUTENBERG EBOOK DRACULA ***',
 '',
 '',
 '',
 '',
 'Produced by Chuck Greif and the Online Distributed',
 'Proofreading Team at http://www.pgdp.net (This file was',
 'produced from images generously made available by The',
 'Internet Archive)',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '                                DRACULA',
 '',
 '',
 '',
 '',
 '',
 '                                DRACULA',
 '',
 '                                  _by_',
 '',
 '                              Bram Stoke

With the RDD `data`, from the previous cell, execute a `flatMap()` for each line in the input and then convert it to lower case, remove the commas, split the words on a space and store in the RDD `words`

In [8]:
words = data.flatMap(lambda x: x.split(',')) \
            .map(lambda x: x.lower())\
            .map(lambda x: (x.replace(',','')))\
            .flatMap(lambda x: x.split(' '))

Execute the command with the action `take()` and retrieve the first 10 words from the `flatMap()`transformation

In [9]:
words.take(10)

['the',
 'project',
 'gutenberg',
 'ebook',
 'of',
 'dracula',
 '',
 'by',
 'bram',
 'stoker']

Perform a classic `map()` to create a tuple where each word has a count of 1

In [10]:
word_tupple=words.map(lambda x:(x,1))
word_tupple.take(10)

[('the', 1),
 ('project', 1),
 ('gutenberg', 1),
 ('ebook', 1),
 ('of', 1),
 ('dracula', 1),
 ('', 1),
 ('by', 1),
 ('bram', 1),
 ('stoker', 1)]

Write the reducing function

In [11]:
word_tupple = word_tupple.reduceByKey(lambda a , b : a+b)
word_tupple.take(5)

[('project', 83),
 ('gutenberg', 24),
 ('ebook', 10),
 ('of', 3719),
 ('dracula', 29)]

Get all words that occur more than once and sort them alphabetically:

In [12]:
word_tupple = word_tupple.filter(lambda x: x[1]>1) \
                                    .sortByKey("desc")
word_tupple.collect()

[('', 20435),
 ('"', 223),
 ('"\'my', 2),
 ('"_17', 4),
 ('"_2', 2),
 ('"_24', 2),
 ('"_25', 4),
 ('"_6', 2),
 ('"_czarina', 2),
 ('"a', 12),
 ('"ah', 18),
 ('"aha!"', 2),
 ('"all', 8),
 ('"am', 2),
 ('"and', 53),
 ('"are', 5),
 ('"arthur!', 2),
 ('"as', 6),
 ('"at', 3),
 ('"be', 2),
 ('"because', 10),
 ('"believe', 3),
 ('"bloofer', 6),
 ('"brave', 2),
 ('"but', 40),
 ('"by', 3),
 ('"can', 4),
 ('"certainly', 3),
 ('"come', 18),
 ('"come!"', 3),
 ('"count', 4),
 ('"darkness', 3),
 ('"dear', 6),
 ('"did', 3),
 ('"do', 20),
 ('"don\'t', 3),
 ('"dr.', 12),
 ('"except', 2),
 ('"fifty', 2),
 ('"for', 9),
 ('"forgive', 4),
 ('"friend', 11),
 ('"go', 8),
 ('"god', 4),
 ('"god!', 2),
 ('"good', 7),
 ('"good!"', 4),
 ('"good-bye', 2),
 ('"have', 6),
 ('"he', 15),
 ('"here', 2),
 ('"his', 3),
 ('"how', 16),
 ('"hush!', 2),
 ('"i', 169),
 ('"i\'ll', 3),
 ('"i\'m', 3),
 ('"if', 9),
 ('"in', 10),
 ('"is', 7),
 ('"it', 32),
 ('"jack', 4),
 ('"john', 2),
 ('"jonathan', 6),
 ('"just', 2),
 ('"let', 8

Now, get all words that occur more than once and sort them by frequency. Select the first 20. Hints : look at the `takeOrdered` action.

In [13]:
word_tupple.takeOrdered(20, lambda x: -x[1])

[('', 20435),
 ('the', 7984),
 ('and', 5840),
 ('i', 4528),
 ('to', 4518),
 ('of', 3719),
 ('a', 2933),
 ('he', 2523),
 ('in', 2515),
 ('that', 2415),
 ('it', 1878),
 ('was', 1848),
 ('as', 1573),
 ('for', 1514),
 ('we', 1499),
 ('is', 1490),
 ('his', 1457),
 ('not', 1350),
 ('with', 1314),
 ('you', 1305)]

### A simple exercice

+ Create a Python collection of 10,000 integers
+ Create a Spark base RDD from that collection
+ Subtract one from each value using map
+ Perform action collect to view results
+ Perform action count to view counts
+ Apply transformation filter and view results with collect

In [14]:
from random import randint
integer_collection=[]
#creates the collection of integers
for i in range(0,10000):
    integer_collection.append(randint(1,100))
#Create Spark RDD
integers = sc.parallelize(integer_collection)
integers.take(10)

[1, 53, 71, 29, 55, 59, 98, 41, 21, 33]

In [15]:
#Removes one from integers, adds a count
integers = integers.flatMap(lambda x:[x-1]) \
                    .map(lambda x: [x,1])
integers.take(10)

[[0, 1],
 [52, 1],
 [70, 1],
 [28, 1],
 [54, 1],
 [58, 1],
 [97, 1],
 [40, 1],
 [20, 1],
 [32, 1]]

In [16]:
#applies collect
integers.collect()

[[0, 1],
 [52, 1],
 [70, 1],
 [28, 1],
 [54, 1],
 [58, 1],
 [97, 1],
 [40, 1],
 [20, 1],
 [32, 1],
 [2, 1],
 [44, 1],
 [88, 1],
 [86, 1],
 [50, 1],
 [28, 1],
 [98, 1],
 [0, 1],
 [90, 1],
 [23, 1],
 [86, 1],
 [12, 1],
 [98, 1],
 [64, 1],
 [88, 1],
 [1, 1],
 [28, 1],
 [83, 1],
 [89, 1],
 [6, 1],
 [95, 1],
 [71, 1],
 [6, 1],
 [73, 1],
 [20, 1],
 [92, 1],
 [13, 1],
 [94, 1],
 [25, 1],
 [38, 1],
 [18, 1],
 [77, 1],
 [74, 1],
 [58, 1],
 [68, 1],
 [4, 1],
 [56, 1],
 [48, 1],
 [96, 1],
 [32, 1],
 [95, 1],
 [61, 1],
 [92, 1],
 [57, 1],
 [30, 1],
 [26, 1],
 [87, 1],
 [86, 1],
 [74, 1],
 [46, 1],
 [63, 1],
 [44, 1],
 [25, 1],
 [0, 1],
 [51, 1],
 [92, 1],
 [53, 1],
 [29, 1],
 [86, 1],
 [54, 1],
 [83, 1],
 [83, 1],
 [0, 1],
 [17, 1],
 [60, 1],
 [85, 1],
 [48, 1],
 [86, 1],
 [24, 1],
 [76, 1],
 [27, 1],
 [70, 1],
 [11, 1],
 [68, 1],
 [90, 1],
 [3, 1],
 [72, 1],
 [91, 1],
 [3, 1],
 [92, 1],
 [29, 1],
 [36, 1],
 [84, 1],
 [47, 1],
 [85, 1],
 [72, 1],
 [92, 1],
 [47, 1],
 [98, 1],
 [49, 1],
 [67, 1],
 

In [17]:
# first re-applies a mapping since a collect was done
integers = integers.map(lambda integer:(integer[0],1))
#counts the frequency
integers_frequency =  integers.reduceByKey(lambda x,y:x+y)
integers_frequency.take(10)

[(0, 118),
 (40, 106),
 (32, 111),
 (88, 110),
 (64, 95),
 (56, 94),
 (48, 99),
 (96, 75),
 (24, 99),
 (72, 129)]

Getting a 100 above means that every number in the range defined with the random function was chosen

In [18]:
#counts number of unique numbers
integers_frequency.count()

100

In [19]:
#filters numbers that are 50 and prints the count
fiftys=integers_frequency.filter(lambda x: x[0]==50)
fiftys.collect()

[(50, 114)]

### An improved WordCount

Print the top 10 most frequent words with their probability of appearance


In [20]:
#gets the total number of words
count_of_words=words.count()
#sends the word count to all RDDs
sc.broadcast(count_of_words)

In [21]:
#computes the probablity, by dividing the value after applying the reduce byt the count of words
top_10_probability=words.map(lambda x:(x,1))\
                .reduceByKey(lambda a , b : a+b)\
                .map(lambda x:[x[0],x[1]*100/count_of_words])\
                .takeOrdered(10, key=lambda x: (-1 * x[1]))

print(top_10_probability)

[['', 11.038427881550943], ['the', 4.312738351177036], ['and', 3.154608212784806], ['i', 2.4459017101865754], ['to', 2.4404999837948207], ['of', 2.008902045093612], ['a', 1.5843263507016843], ['he', 1.3628555686397372], ['in', 1.3585341875263335], ['that', 1.3045169236087855]]


Get rid of special characters (.,:!?')


In [22]:
import re
#getting rid of special characters
words_without_characters = data.map(lambda x: x.split(' '))\
                                .flatMap(lambda x: x)\
                                .map(lambda x: (re.sub('\W','', x).lower()))\
                                .filter(lambda x:x!='')
#counting the words again
count_of_words=words_without_characters.count()
sc.broadcast(count_of_words)
#doing the probability again
top_10_probability=words_without_characters.map(lambda x:(x,1))\
                .reduceByKey(lambda a , b : a+b)\
                .map(lambda x:[x[0],x[1]*100/count_of_words])\
                .takeOrdered(10, key=lambda x: (-1 * x[1]))
print(top_10_probability)

[['the', 4.905266243998974], ['and', 3.6012264692588656], ['i', 2.877438584918337], ['to', 2.772993244646413], ['of', 2.2831384908564516], ['a', 1.8067211492652178], ['in', 1.5599613979795017], ['he', 1.5520211089529812], ['that', 1.4988822516216513], ['it', 1.3058721491308438]]


Identify the transformations and the actions in your script


The transformations include the mapping, the flat mapping and the filtering
the actions are the reduceByKey and the take ordered

How many times are the transformations evaluated? (Hint: it depends)


The transformation is only evaluated after an action has been instiguated.
Each time a new action is applied, the transformation is applied again, as the results of the transformation are not stored in memory. 

Can you reduce this number? (Hint: check out "persist")

Selecting the persist would result in the transformations being saved into memory. Hence, when a new action is applied, the transformed data is utilized.

In [23]:
sc.stop()

## Part 3 : Spark SQL and dataframes

In this part, you will explore Spark DataFrames and the SQL Context. In particular, we will work on a database that contains a sample of the world population by working on data that comes from [pplapi](http://pplapi.com/). The file [agents.json](./SparkData/agents.json) is a file that was extracted from this api using the following command :


`wget https://s3-eu-west-1.amazonaws.com/course.oc-static.com/courses/4297166/agents.json`

In [24]:
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

df = spark.read.json("./SparkData/agents.json")

Print the 5 first lines of the dataframe `df`

In [25]:
df.head(5)

[Row(country_name='China', id=227417393, latitude=33.15219798270325, longitude=100.85840672174572, sex='Male'),
 Row(country_name='Haiti', id=6821129477, latitude=19.325567983697297, longitude=-72.43795260265814, sex='Female'),
 Row(country_name='India', id=2078667700, latitude=23.645271492037235, longitude=80.85636526088884, sex='Female'),
 Row(country_name='China', id=477556555, latitude=33.45864668881662, longitude=93.33604038078953, sex='Female'),
 Row(country_name='India', id=1379059984, latitude=28.816938290678692, longitude=80.7728698035823, sex='Female')]

Write the spark instructions that enable to display the number of French agents.

In [26]:
df.filter(df.country_name=='France').count()

94

Write the spark instructions that enable to display the number of Indian female agents.

In [27]:
df.filter((df.country_name=='India') & (df.sex=='Female')).count()

828

Using the notion of temporary view (function `createTempView`), create a temporary view associated to the dataframe `df`. 

In [28]:
catalog = spark.catalog

#if table already generated, drops it
try:
    catalog.dropTempView("agents_table")
except:
    None


df.createTempView("agents_table")

#checks if temp_table is cached
df.cache()
catalog.isCached("agents_table")

True

Write some SQL query on the resulting table as shown in the example below.

In [29]:
spark.sql("SELECT country_name,id FROM agents_table ORDER BY id DESC LIMIT 10").show()

+-----------------+----------+
|     country_name|        id|
+-----------------+----------+
| French Polynesia|7170821229|
|       Cabo Verde|7167692449|
|         Suriname|7166451460|
|         Suriname|7166235088|
|            Macau|7166034642|
|       Montenegro|7164357515|
|Equatorial Guinea|7163867872|
|           Bhutan|7163256789|
|           Bhutan|7163004645|
|           Bhutan|7162877973|
+-----------------+----------+



In [30]:
spark.sql("SELECT country_name, sex, count(sex) as number_of_agents FROM agents_table group by country_name, sex having country_name = 'India'").show()

+------------+------+----------------+
|country_name|   sex|number_of_agents|
+------------+------+----------------+
|       India|  Male|             905|
|       India|Female|             828|
+------------+------+----------------+



In [31]:
spark.stop()
sc.stop()

You can also create a dataframe from an existing RDD as shown on the example below

In [32]:
sc=pyspark.SparkContext("local[*]","Dataframe examples in Spark")
sqlContext = pyspark.SQLContext(sc)

In [33]:
documents_rdd = sc.parallelize([
        [1, 'cats are cute', 0],
        [2, 'dogs are playfull', 0],
        [3, 'lions are big', 1],
        [4, 'cars are fast', 1]])
users_rdd = sc.parallelize([
        [0, 'Alice', 20],
        [1, 'Bob', 23],
        [2, 'Charles', 32]])

In [34]:
documents_df = documents_rdd.toDF(['doc_id', 'text', 'user_id'])
users_df = users_rdd.toDF(['user_id', 'name', 'age'])

# printing the inferred schema for documents
documents_df.printSchema()


root
 |-- doc_id: long (nullable = true)
 |-- text: string (nullable = true)
 |-- user_id: long (nullable = true)



Some functions can be apply to a column or to different columns. Here, we compute the age avarage in the dataframe `users_df`. 

In [35]:
from pyspark.sql import functions as fn

user_age_df = users_df.select(fn.avg('age')).show()

+--------+
|avg(age)|
+--------+
|    25.0|
+--------+




Compute the max of age in the dataframe `users_df`. 

In [36]:
user_age_df = users_df.select(fn.max('age')).show()

+--------+
|max(age)|
+--------+
|      32|
+--------+



Join (as in SQL) the two dataframes `users_df` and `documents_df`

In [37]:
inner_join = users_df.join(documents_df, users_df.user_id == documents_df.user_id)
inner_join.show()

+-------+-----+---+------+-----------------+-------+
|user_id| name|age|doc_id|             text|user_id|
+-------+-----+---+------+-----------------+-------+
|      0|Alice| 20|     1|    cats are cute|      0|
|      0|Alice| 20|     2|dogs are playfull|      0|
|      1|  Bob| 23|     3|    lions are big|      1|
|      1|  Bob| 23|     4|    cars are fast|      1|
+-------+-----+---+------+-----------------+-------+



Outer Join (Left) (as in SQL) the two dataframes `users_df` and `documents_df`

In [38]:
outer_lef_join = users_df.join(documents_df, users_df.user_id == documents_df.user_id,how='left')
outer_lef_join.show()

+-------+-------+---+------+-----------------+-------+
|user_id|   name|age|doc_id|             text|user_id|
+-------+-------+---+------+-----------------+-------+
|      0|  Alice| 20|     1|    cats are cute|      0|
|      0|  Alice| 20|     2|dogs are playfull|      0|
|      1|    Bob| 23|     3|    lions are big|      1|
|      1|    Bob| 23|     4|    cars are fast|      1|
|      2|Charles| 32|  null|             null|   null|
+-------+-------+---+------+-----------------+-------+



## Part 4 : MLlib - Spark for data analysis and machine learning

In this part, we will see how to create a text classifiation application with Spark. We will use some data from the newsgroups [Usenet](https://en.wikipedia.org/wiki/Usenet_newsgroup) and the objective is to predict the topic of the news.
 
The train data is [here](./SparkData/20ng-train-all-terms.txt) and the test data [here](./SparkData/20ng-test-all-terms.txt).
The are obtained using the following commands: 
`wget http://ana.cachopo.org/datasets-for-single-label-text-categorization/20ng-train-all-terms.txt`
and
`wget http://ana.cachopo.org/datasets-for-single-label-text-categorization/20ng-test-all-terms.txt`

Load these data as some RDDs and translate them in DataFrames.

In [39]:
sc.stop()

findspark.init()
from pyspark.ml.feature import CountVectorizer
from pyspark.sql.functions import split


sc=pyspark.SparkContext("local[*]","machine learning in Spark")
sqlContext = pyspark.SQLContext(sc)

def load_dataframe(path):
    rdd= sc.textFile(path)
    rdd = rdd.map(lambda x: x.split('\t'))
    df = rdd.toDF(['label','words'])
    df = df.withColumn("words", split("words","\s+"))
    return df 

train_data = load_dataframe("./SparkData/train-all-terms.txt")
test_data = load_dataframe("./SparkData/test-all-terms.txt")
train_data.show()
test_data.show()

+-----------+--------------------+
|      label|               words|
+-----------+--------------------+
|alt.atheism|[alt, atheism, fa...|
|alt.atheism|[alt, atheism, fa...|
|alt.atheism|[re, gospel, dati...|
|alt.atheism|[re, university, ...|
|alt.atheism|[re, soc, motss, ...|
|alt.atheism|[re, a, visit, fr...|
|alt.atheism|[re, political, a...|
|alt.atheism|[re, an, anecdote...|
|alt.atheism|[re, political, a...|
|alt.atheism|[re, pompous, ass...|
|alt.atheism|[re, pompous, ass...|
|alt.atheism|[re, keith, schne...|
|alt.atheism|[re, keith, schne...|
|alt.atheism|[re, political, a...|
|alt.atheism|[re, political, a...|
|alt.atheism|[re, political, a...|
|alt.atheism|[re, don, t, more...|
|alt.atheism|[re, ancient, isl...|
|alt.atheism|[re, political, a...|
|alt.atheism|[re, there, must,...|
+-----------+--------------------+
only showing top 20 rows

+-----------+--------------------+
|      label|               words|
+-----------+--------------------+
|alt.atheism|[re, about, the,

A first step is to represent our data, i.e. the messages in the form of a bag-of-word representation using the spark method `CountVectorizer` documented [here](https://spark.apache.org/docs/2.1.0/ml-features.html#countvectorizer)

In [40]:
cv = CountVectorizer(inputCol="words", outputCol="features", minDF=2.0)
model_transform = cv.fit(train_data)
vectorized_train = model_transform.transform(train_data)
vectorized_train.show(truncate=True)

vectorized_test = model_transform.transform(test_data)
vectorized_test.show(truncate=True)


+-----------+--------------------+--------------------+
|      label|               words|            features|
+-----------+--------------------+--------------------+
|alt.atheism|[alt, atheism, fa...|(42281,[0,1,2,3,4...|
|alt.atheism|[alt, atheism, fa...|(42281,[0,1,2,3,4...|
|alt.atheism|[re, gospel, dati...|(42281,[0,1,2,3,4...|
|alt.atheism|[re, university, ...|(42281,[0,1,2,3,4...|
|alt.atheism|[re, soc, motss, ...|(42281,[0,1,2,3,4...|
|alt.atheism|[re, a, visit, fr...|(42281,[0,1,2,3,4...|
|alt.atheism|[re, political, a...|(42281,[0,1,3,4,6...|
|alt.atheism|[re, an, anecdote...|(42281,[0,1,2,3,4...|
|alt.atheism|[re, political, a...|(42281,[0,1,2,3,4...|
|alt.atheism|[re, pompous, ass...|(42281,[0,4,5,11,...|
|alt.atheism|[re, pompous, ass...|(42281,[0,1,2,3,4...|
|alt.atheism|[re, keith, schne...|(42281,[0,1,2,3,4...|
|alt.atheism|[re, keith, schne...|(42281,[0,1,2,3,4...|
|alt.atheism|[re, political, a...|(42281,[0,1,2,3,4...|
|alt.atheism|[re, political, a...|(42281,[0,1,2,

Displayong of the distint labels in the datasets

In [41]:
train_data.select("label").distinct().sort("label").show(truncate=False)

+------------------------+
|label                   |
+------------------------+
|alt.atheism             |
|comp.graphics           |
|comp.os.ms-windows.misc |
|comp.sys.ibm.pc.hardware|
|comp.sys.mac.hardware   |
|comp.windows.x          |
|misc.forsale            |
|rec.autos               |
|rec.motorcycles         |
|rec.sport.baseball      |
|rec.sport.hockey        |
|sci.crypt               |
|sci.electronics         |
|sci.med                 |
|sci.space               |
|soc.religion.christian  |
|talk.politics.guns      |
|talk.politics.mideast   |
|talk.politics.misc      |
|talk.religion.misc      |
+------------------------+



We will now apply a [NaiveBayes](https://spark.apache.org/docs/latest/ml-classification-regression.html#naive-bayes) classifier to our problem. 
Take the time to read the doc and apply it to our problem. You will first have to associate a number to each label. You can use the [`String Indexer`](https://spark.apache.org/docs/2.1.0/ml-features.html#stringindexer) function of pyspark for that.


In [42]:
# TO DO - Transformation of the label into a number

from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import IndexToString, StringIndexer
from pyspark.sql.functions import monotonically_increasing_id 


#combines them now, splits them back later, for the Baysian to work
#full_set = vectorized_train.union(vectorized_test)

indexer = StringIndexer(inputCol="label", outputCol="categoryIndex")
model = indexer.fit(vectorized_train)

indexed_train = model.transform(vectorized_train)
indexed_train = indexed_train.drop('label')
indexed_train = indexed_train.drop('words')
indexed_train = indexed_train.withColumnRenamed("categoryIndex", "label")


indexed_test = model.transform(vectorized_test)
indexed_test = indexed_test.drop('label')
indexed_test = indexed_test.drop('words')
indexed_test = indexed_test.withColumnRenamed("categoryIndex", "label")

In [43]:
# TO DO - Application of the NaivesBayes models
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

nb = NaiveBayes(smoothing=1.0, modelType="multinomial")
model = nb.fit(indexed_train)


In [44]:
# TO DO - Application of the trained model to the test dataset and displaying the prediction compared to the real value for the 20 first messages
predictions= model.transform(indexed_test)
predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(42281,[0,1,2,3,4...| 17.0|[-1054.8071325254...|[3.21314151219117...|      17.0|
|(42281,[0,1,2,3,4...| 17.0|[-4719.7244858947...|[6.94408203926236...|      17.0|
|(42281,[0,1,2,3,4...| 17.0|[-2733.3896730016...|[1.23477383768239...|      17.0|
|(42281,[0,1,2,3,4...| 17.0|[-1545.4076738797...|[3.48453364935401...|      17.0|
|(42281,[0,1,2,3,4...| 17.0|[-3788.3670782352...|[4.34053021737716...|      17.0|
|(42281,[0,1,2,3,4...| 17.0|[-2365.6678130709...|[4.76445704428970...|      17.0|
|(42281,[0,1,2,3,4...| 17.0|[-2617.4514399846...|[1.12856422263077...|      19.0|
|(42281,[0,1,4,5,6...| 17.0|[-614.61643061758...|[8.13527378809278...|      17.0|
|(42281,[0,1,4,5,6...| 17.0|[-554.66325821277...|[5.57905592463520...|      17.0|
|(42281,[0,1,4,5

The Evaluation of the learned model can be done by using the [evaluation](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html) module of MLlib. Print the accuracy of the obtained model.

In [45]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.8100425079702445
